In [ ]:
import sqlite3
import csv

def init_db():
    with sqlite3.connect('contacts.db') as conn:
        conn.execute('''
            CREATE TABLE IF NOT EXISTS Contacts (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                phone TEXT UNIQUE NOT NULL
            )
        ''')

def reset_table():
    with sqlite3.connect('contacts.db') as conn:
        conn.execute("DELETE FROM Contacts")
        conn.execute("DELETE FROM sqlite_sequence WHERE name='Contacts'")
    print("Таблица сброшена.")

def add_contact_manual():
    name = input("Имя: ").strip()
    number = input("Телефон: ").strip()
    try:
        with sqlite3.connect('contacts.db') as conn:
            conn.execute("INSERT INTO Contacts (name, phone) VALUES (?, ?)", (name, number))
        print("Контакт добавлен.")
    except sqlite3.IntegrityError:
        print("Ошибка: этот номер уже есть в базе.")

def load_from_file(file_path):
    try:
        with open(file_path, encoding='utf-8-sig') as f:
            reader = csv.reader(f)
            with sqlite3.connect('contacts.db') as conn:
                for i, row in enumerate(reader, start=1):
                    if len(row) < 2:
                        print(f"Строка {i} пропущена: не хватает данных.")
                        continue
                    name, phone = row[0].strip(), row[1].strip().replace(' ', '')
                    if not name or not phone:
                        print(f"Строка {i} пропущена: пустые поля.")
                        continue
                    try:
                        conn.execute("INSERT INTO Contacts (name, phone) VALUES (?, ?)", (name, phone))
                    except sqlite3.IntegrityError:
                        print(f"Дубликат найден, пропущено: {phone}")
        print("Импорт завершён.")
    except FileNotFoundError:
        print("Файл не найден.")

def edit_contact():
    field = input("Что хотите изменить (name/phone): ").strip()
    if field not in ['name', 'phone']:
        print("Недопустимое поле.")
        return
    current = input(f"Текущее значение {field}: ")
    updated = input("Новое значение: ")
    with sqlite3.connect('contacts.db') as conn:
        cursor = conn.execute(f"UPDATE Contacts SET {field} = ? WHERE {field} = ?", (updated, current))
        if cursor.rowcount:
            print("Контакт обновлён.")
        else:
            print("Контакт не найден.")

def show_contacts():
    search_field = input("Фильтровать по (name/phone или пропустить): ").strip()
    value = ""
    if search_field in ['name', 'phone']:
        value = input(f"Значение для {search_field}: ").strip()

    with sqlite3.connect('contacts.db') as conn:
        if search_field and value:
            rows = conn.execute(f"SELECT name, phone FROM Contacts WHERE {search_field} = ?", (value,)).fetchall()
        else:
            rows = conn.execute("SELECT name, phone FROM Contacts").fetchall()

        if rows:
            print("\nСписок контактов:")
            for i, (name, phone) in enumerate(rows, 1):
                print(f"{i}. {name} — {phone}")
        else:
            print("Контактов нет.")

def remove_contact():
    field = input("Удалить по (name/phone): ").strip()
    if field not in ['name', 'phone']:
        print("Неверное поле.")
        return
    value = input("Введите значение для удаления: ").strip()
    with sqlite3.connect('contacts.db') as conn:
        cursor = conn.execute(f"DELETE FROM Contacts WHERE {field} = ?", (value,))
        if cursor.rowcount:
            print("Контакт удалён.")
        else:
            print("Совпадений не найдено.")

def main():
    init_db()
    while True:
        print("\n--- Телефонный справочник ---")
        print("1. Добавить контакт вручную")
        print("2. Импортировать из CSV")
        print("3. Обновить контакт")
        print("4. Показать контакты")
        print("5. Удалить контакт")
        print("6. Очистить таблицу")
        print("0. Выйти")

        option = input("Выбор: ")
        match option:
            case "1": add_contact_manual()
            case "2":
                path = input("Путь к CSV-файлу: ")
                load_from_file(path)
            case "3": edit_contact()
            case "4": show_contacts()
            case "5": remove_contact()
            case "6": reset_table()
            case "0":
                print("Завершение программы.")
                break
            case _: print("Неверный ввод.")

if __name__ == "__main__":
    main()